# 1. experiment design

## 1.0 function and parameter

In [138]:
import pandas as pd 
import matplotlib.pyplot as plt
import numpy as np
import argparse
import math
from scipy import stats
import os
from tqdm.notebook import tqdm

#let PID system adaptively changes bids.
def PID_sys(current_cpc_, target_cpc_, current_ecpc_):
    #Here we use a simplified version of PID: (1) k_I=p_D=0, (2) set current_ecpc = current_cpc
    #which means this simplified PID use an exponential smoothing to update bids
    
    k_p = 0.4
    
    #equivalent version:
    #error = target_cpc_ - current_ecpc_
    #current_cpc_updated = current_cpc_ + k_p*error
    current_cpc_updated = k_p * target_cpc_ + (1-k_p) * current_cpc_
    return current_cpc_updated


#Define context generating function.
gender_pdf = np.array([0.5, 0.5])#marginal distribution of gender. [male, female]
location_pdf = np.array([0.22, 0.46, 0.32])#marginal distribution of city level. [large city, medium city, small city]
age_seg_pdf = np.array([0.46, 0.34, 0.20])#marginal distribution of age. [small age, middle age, large age]
dim_context  = int(np.shape(gender_pdf)[0]+np.shape(location_pdf)[0]+np.shape(age_seg_pdf)[0] + 1)
def context_generate(gender_ratio = gender_pdf, loc_ratio = location_pdf, age_ratio = age_seg_pdf):
    a = np.random.multinomial(1, gender_pdf, size=1)
    b = np.random.multinomial(1, location_pdf, size=1)
    c = np.random.multinomial(1, age_ratio, size=1)
    return np.concatenate((a, b, c, 1), axis=None)


#Load sampled data and define ad-campaign generating function.
#load joint information of ctr fixed effect coefficient, and target bid (CPC)
ctr_bid_np = np.load('ctr_bid_data.npy')
#note the first 200 rows are marure ads and last 100 rows are new ads
def generate_ad(n, new_raito = 0.30, dim_context = dim_context, data = ctr_bid_np):
    #n is the total number of ads, new_raito is ratio of new ads
    num_new_ad = int(n*new_raito)
    num_mature_ad = n - num_new_ad
    sampled_ctr = data[:, 0]
    sampled_cpc = data[:, 1]
    
    #initialize output data: 
    is_new_index = np.zeros(n)
    target_cpc = np.zeros(n)
    coefficient_ctr_model = np.zeros([n, dim_context])
    
    #generate bid, true CTR model
    for i in range(n):
        index = 0
        if i < num_mature_ad:
            is_new_index[i] = 0
            index = np.random.choice(np.arange(0,200))
        else:
            is_new_index[i] = 1
            index = np.random.choice(np.arange(200,300)) 
        target_cpc[i] = sampled_cpc[index]
        coefficient_ctr_model[i, 0:-1] = np.random.uniform(-0.05,0.05,dim_context-1)
        coefficient_ctr_model[i, -1] = sampled_ctr[index]
    return np.int32(is_new_index), target_cpc, coefficient_ctr_model

# hyperparameters settings. Notice that this system is based on first price auction and CPC,
import sys
sys.argv=['']
del sys
parser = argparse.ArgumentParser()

#Default cold start parameters - the cold start value = reward*I(clicks>=target_click)
parser.add_argument('--total_ads', type=int, default=1000, help='total number of ads for simulation')
parser.add_argument('--new_ad_ratio', type=float, default=0.3, help='ratio of new ads')
parser.add_argument('--total_pv', type=int, default=1000000, help='loaded total page views in one cold start horizon for simulation')
parser.add_argument('--reward', type=float, default=1000000, help='reward of cold start')
parser.add_argument('--delta',  type=float, default=200000, help='delta reward of treated group')
parser.add_argument('--target_click', type=int, default=1000, help='target click')
parser.add_argument('--alpha', type=float, default=1e-3, help='target click/total_pv')
#If one wants to change total_ads, it is better to proportionally change total_pv to balance the two-side market

#Others parameters
parser.add_argument('--budget_limit', type=int, default=5e5, help='set an uniform budget limit on ads')
parser.add_argument('--stepsize', type=float, default=30, help='step size of DMD')
parser.add_argument('--PID_cycle', type=int, default=100000, help='after every # pvs, update PID')

#Parameters about experiments - sampling rate
parser.add_argument('--ad_treatment_ratio', type=float, default=0.4, help='range from 0 to 0.5')
parser.add_argument('--pv_treatment_ratio', type=float, default=0.4, help='range from 0 to 0.5')

#Parameters about model - learning rate
parser.add_argument('--l_rate', type=float, default=0.001, help='learning rage')

_StoreAction(option_strings=['--l_rate'], dest='l_rate', nargs=None, const=None, default=0.001, type=<class 'float'>, choices=None, required=False, help='learning rage', metavar=None)

## 1.1 all-control

In [139]:
def all_control(opt, is_new_index, target_bid_price, coefficient_ctr_model):
    
    #Initialize the simulation
    target_cpc = np.copy(target_bid_price)
    total_clicks = np.zeros(opt.total_ads)#accumulated number of clicks
    total_impressions = np.zeros(opt.total_ads)#accumulated number of impressions
    total_cost = np.zeros(opt.total_ads)#accumulated cost of ads
    coefficient_ctr_pred_model = np.zeros([opt.total_ads, dim_context]) 

    #assume for mature ads, we know exactly the true CTR model
    for i in range(opt.total_ads):
        if is_new_index[i]==0:
            coefficient_ctr_pred_model[i] = coefficient_ctr_model[i]
        else:
            #for new ads, initialize average of real CTR and mean CTR = 0.067 (over all new ads)
            #to mimic real situation, reco sys has partial prior knowledge of new ads
            #typically initialize by avergae over all ads with the same industry
            coefficient_ctr_pred_model[i, -1] = 0.5*coefficient_ctr_model[i, -1] + 0.5*0.067
    new_ad_index = np.where(is_new_index==1)[0]

    #Initialize PID system
    current_cpc = np.zeros(opt.total_ads)
    for i in range(opt.total_ads):
        if is_new_index[i]==1:
            current_cpc[i] = target_cpc[i] + opt.reward/opt.target_click

    #simulation starts
    for i in tqdm(range(opt.total_pv), desc="All control experiment"):

        ##########################generate context##########################
        contexts = context_generate()

        ##########################calculate eCPM and Auction##########################
        #calculate pCTR and eCPM = pCTR*bid
        current_bid_np = np.zeros(opt.total_ads)
        pCTR = np.maximum(0, np.minimum(np.matmul(coefficient_ctr_pred_model, contexts), 1))

        current_bid_np = target_cpc*(1-is_new_index) + current_cpc*is_new_index#if using PID
        current_bid_np -= 10000*np.greater(total_cost, opt.budget_limit*np.ones(opt.total_ads))
        ecpm = pCTR * current_bid_np
        #ranking
        max_index = np.argmax(ecpm, axis=0)

        ##########################click feedback and cost reduction##########################  
        total_impressions[max_index] += 1
        #get the feedback
        real_ctr = np.matmul(coefficient_ctr_model[max_index], contexts)
        is_clicked = np.random.binomial(1, max(0, min(real_ctr, 1)))
        if is_clicked == 1:#if clicked
            total_clicks[max_index] += 1
            total_cost[max_index] += target_cpc[max_index]

        ##########################update pCTR prediction model##########################
        if is_new_index[max_index]>0:#only update ctr model for new ads since we assume mature ad prediction is accurate
            coefficient_ctr_pred_model[max_index] += opt.l_rate*(is_clicked-pCTR[max_index])*contexts#SGD for linear models

        ##########################update PID##########################
        #using PID system to update current bid
        if i%opt.PID_cycle==(opt.PID_cycle-1):
            current_ecpc = total_cost/(total_clicks+0.01)#compute ecpc, +0.01 to avoid 0/0=NaN
            current_cpc = PID_sys(current_cpc, target_cpc, current_ecpc)
        current_cpc = np.less(total_clicks, opt.target_click)*current_cpc + (1- np.less(total_clicks, opt.target_click))*target_cpc
    
    
    #Output simulation results
    print('================================================')
    rev_new_avg = np.sum(total_cost*is_new_index)/( np.sum(is_new_index)*opt.total_pv)
    rev_new_tot = np.sum(total_cost*is_new_index)
    print('Average Revenue in new:', rev_new_avg)
    print('Total   Revenue in new:', rev_new_tot)
    
    clicks_new_avg = np.sum(total_clicks*is_new_index)/( np.sum(is_new_index)*opt.total_pv)
    clicks_new_tot = np.sum(total_clicks*is_new_index)
    print('Average clicks in new:', clicks_new_avg)
    print('Total   clicks in new:', clicks_new_tot)
    
    coldstart_rate = (total_clicks*is_new_index>=opt.target_click).sum()/is_new_index.sum()#potential outcome in control, take Cold Start Success Rate as example
    print('Cold Start Success Rate:', coldstart_rate)
    print('================================================')
    
    return (rev_new_avg, rev_new_tot, 
            clicks_new_avg, clicks_new_tot, 
            coldstart_rate)

## 1.2 all-treat

In [140]:
def all_treat(opt, is_new_index, target_bid_price, coefficient_ctr_model):
    #Initialize the simulation
    target_cpc = np.copy(target_bid_price)
    total_clicks = np.zeros(opt.total_ads)#accumulated number of clicks
    total_impressions = np.zeros(opt.total_ads)#accumulated number of impressions
    total_cost = np.zeros(opt.total_ads)#accumulated cost of ads
    coefficient_ctr_pred_model = np.zeros([opt.total_ads, dim_context]) 

    #assume for mature ads, we know exactly the true CTR model
    for i in range(opt.total_ads):
        if is_new_index[i]==0:
            coefficient_ctr_pred_model[i] = coefficient_ctr_model[i]
        else:
            #for new ads, initialize average of real CTR and mean CTR = 0.067 (over all new ads)
            #to mimic real situation, reco sys has partial prior knowledge of new ads
            #typically initialize by avergae over all ads with the same industry
            coefficient_ctr_pred_model[i, -1] = 0.5*coefficient_ctr_model[i, -1] + 0.5*0.067
    new_ad_index = np.where(is_new_index==1)[0]

    #Initialize PID system
    current_cpc = np.zeros(opt.total_ads)
    for i in range(opt.total_ads):
        if is_new_index[i]==1:
            current_cpc[i] = target_cpc[i] + (opt.reward+opt.delta)/opt.target_click

    #simulation starts
    for i in tqdm(range(opt.total_pv), desc="All treat experiment"):

        ##########################generate context##########################
        contexts = context_generate()

        ##########################calculate eCPM and Auction##########################
        #calculate pCTR and eCPM = pCTR*bid
        current_bid_np = np.zeros(opt.total_ads)
        pCTR = np.maximum(0, np.minimum(np.matmul(coefficient_ctr_pred_model, contexts), 1))

        current_bid_np = target_cpc*(1-is_new_index) + current_cpc*is_new_index#if using PID
        current_bid_np -= 10000*np.greater(total_cost, opt.budget_limit*np.ones(opt.total_ads))
        ecpm = pCTR * current_bid_np
        #ranking
        max_index = np.argmax(ecpm, axis=0)

        ##########################click feedback and cost reduction##########################  
        total_impressions[max_index] += 1
        #get the feedback
        real_ctr = np.matmul(coefficient_ctr_model[max_index], contexts)
        is_clicked = np.random.binomial(1, max(0, min(real_ctr, 1)))
        if is_clicked == 1:#if clicked
            total_clicks[max_index] += 1
            total_cost[max_index] += target_cpc[max_index]

        ##########################update pCTR prediction model##########################
        if is_new_index[max_index]>0:#only update ctr model for new ads since we assume mature ad prediction is accurate
            coefficient_ctr_pred_model[max_index] += opt.l_rate*(is_clicked-pCTR[max_index])*contexts#SGD for linear models


        ##########################update PID##########################
        #using PID system to update current bid
        if i%opt.PID_cycle==(opt.PID_cycle-1):
            current_ecpc = total_cost/(total_clicks+0.01)#compute ecpc, +0.01 to avoid 0/0=NaN
            current_cpc = PID_sys(current_cpc, target_cpc, current_ecpc)
        current_cpc = np.less(total_clicks, opt.target_click)*current_cpc + (1- np.less(total_clicks, opt.target_click))*target_cpc
        
    #Output simulation results
    print('================================================')
    rev_new_avg = np.sum(total_cost*is_new_index)/( np.sum(is_new_index)*opt.total_pv)
    rev_new_tot = np.sum(total_cost*is_new_index)
    print('Average Revenue in new:', rev_new_avg)
    print('Total   Revenue in new:', rev_new_tot)
    
    clicks_new_avg = np.sum(total_clicks*is_new_index)/( np.sum(is_new_index)*opt.total_pv)
    clicks_new_tot = np.sum(total_clicks*is_new_index)
    print('Average clicks in new:', clicks_new_avg)
    print('Total   clicks in new:', clicks_new_tot)
    
    coldstart_rate = (total_clicks*is_new_index>=opt.target_click).sum()/is_new_index.sum()#potential outcome in control, take Cold Start Success Rate as example
    print('Cold Start Success Rate:', coldstart_rate)
    print('================================================')
    return (rev_new_avg, rev_new_tot, 
            clicks_new_avg, clicks_new_tot, 
            coldstart_rate)

## 1.3 ad-side exp

In [141]:
def ad_exp(opt, is_new_index, target_bid_price, coefficient_ctr_model):
    print('Ad-side experiment')
    print('# of new ads in treatment condition', int(opt.total_ads*opt.new_ad_ratio*opt.ad_treatment_ratio))
    print('# of new ads in control condition', int(opt.total_ads*opt.new_ad_ratio*opt.ad_treatment_ratio))

    #randomly assign new ads into treatment and control groups
    is_new_treatment = np.zeros(opt.total_ads)
    is_new_control = np.zeros(opt.total_ads)

    new_ad_index = np.where(is_new_index==1)[0]
    treatment_new_ad_index = np.random.choice(new_ad_index, int(opt.total_ads*opt.new_ad_ratio*opt.ad_treatment_ratio), replace=False)

    control_new_ad_index = np.random.choice(np.setdiff1d(new_ad_index, treatment_new_ad_index), 
                                              int(opt.total_ads*opt.new_ad_ratio*opt.ad_treatment_ratio), replace=False)

    for i in range(int(opt.total_ads*opt.new_ad_ratio*opt.ad_treatment_ratio)):
        is_new_treatment[treatment_new_ad_index[i]] = 1
        is_new_control[control_new_ad_index[i]] = 1

    #initialization
    target_cpc = np.copy(target_bid_price)
    total_clicks = np.zeros(opt.total_ads)#accumulated number of clicks
    total_impressions = np.zeros(opt.total_ads)#accumulated number of impressions
    total_cost = np.zeros(opt.total_ads)#accumulated cost of ads
    coefficient_ctr_pred_model = np.zeros([opt.total_ads, dim_context]) 
    for i in range(opt.total_ads):
        if is_new_index[i]==0:
            coefficient_ctr_pred_model[i] = coefficient_ctr_model[i]
        else:
            coefficient_ctr_pred_model[i, -1] = 0.5*coefficient_ctr_model[i, -1]+0.5*0.067
    current_cpc = np.zeros(opt.total_ads)

    for i in range(opt.total_ads):
        if is_new_control[i] == 1:
            #new control ads
            current_cpc[i] = target_cpc[i] + opt.reward/opt.target_click
        if is_new_treatment[i] == 1:
            #new treatment ads
            current_cpc[i] = target_cpc[i] + (opt.reward + opt.delta) /opt.target_click

        if is_new_index[i] == 1 and is_new_control[i] != 1 and is_new_treatment[i] != 1:
            #other control ads
            current_cpc[i] = target_cpc[i] + opt.reward/opt.target_click

    #simulation starts
    for i in tqdm(range(opt.total_pv), desc="ad side experiment"):

        ##########################generate context##########################
        contexts = context_generate()

        ##########################calculate eCPM and Auction##########################
        #calculate pCTR and eCPM = pCTR*bid
        current_bid_np = np.copy(target_cpc)
        pCTR = np.maximum(0, np.minimum(np.matmul(coefficient_ctr_pred_model, contexts), 1))
        current_bid_np = target_cpc*(1-is_new_index) + current_cpc*is_new_index

        current_bid_np -= 10000*np.greater(total_cost, opt.budget_limit*np.ones(opt.total_ads))
        ecpm = pCTR * current_bid_np
        #ranking
        max_index = np.argmax(ecpm, axis=0)

        ##########################click feedback and cost reduction##########################  
        total_impressions[max_index] += 1
        #get the feedback
        real_ctr = np.matmul(coefficient_ctr_model[max_index], contexts)
        is_clicked = np.random.binomial(1, max(0, min(real_ctr, 1)))
        if is_clicked == 1:#if clicked
            total_clicks[max_index] += 1
            total_cost[max_index] += target_cpc[max_index]

        ##########################update pCTR prediction model##########################
        if is_new_index[max_index]>0:#only update ctr model for new ads since we assume mature ad prediction is accurate
            coefficient_ctr_pred_model[max_index] += opt.l_rate*(is_clicked-pCTR[max_index])*contexts#SGD for linear models

        ##########################update lambda or PID##########################
        #using PID system to update current bid
        if i%opt.PID_cycle==(opt.PID_cycle-1):
            current_ecpc = total_cost/(total_clicks+0.01)#compute ecpc, +0.01 to avoid 0/0=NaN
            current_cpc = PID_sys(current_cpc, target_cpc, current_ecpc)
        current_cpc = np.less(total_clicks, opt.target_click)*current_cpc + (1- np.less(total_clicks, opt.target_click))*target_cpc
        
    #Simulation results
    print('================================================')
    rev_new_avg_treat_group = np.sum(total_cost*is_new_treatment)/(np.sum(is_new_treatment)*opt.total_pv)
    rev_new_avg_control_group = np.sum(total_cost*is_new_control)/(np.sum(is_new_control)*opt.total_pv)
    rev_new_tot_treat_group = np.sum(total_cost*is_new_treatment)
    rev_new_tot_control_group = np.sum(total_cost*is_new_control)
    print('Average Revenue in new of treat   group:', rev_new_avg_treat_group)
    print('Average Revenue in new of control group:', rev_new_avg_control_group)
    print('Total   Revenue in new of treat   group:', rev_new_tot_treat_group)
    print('Total   Revenue in new of control group:', rev_new_tot_control_group)
    
    clicks_new_avg_treat_group = np.sum(total_clicks*is_new_treatment)/(np.sum(is_new_treatment)*opt.total_pv)
    clicks_new_avg_control_group = np.sum(total_clicks*is_new_control)/(np.sum(is_new_control)*opt.total_pv)
    clicks_new_tot_treat_group = np.sum(total_clicks*is_new_treatment)
    clicks_new_tot_control_group = np.sum(total_clicks*is_new_control)
    print('Average clicks in new of treat   group:', clicks_new_avg_treat_group)
    print('Average clicks in new of control group:', clicks_new_avg_control_group)
    print('Total   clicks in new of treat   group:', clicks_new_tot_treat_group)
    print('Total   clicks in new of control group:', clicks_new_tot_control_group)
    
    coldstart_rate_treat_group = (total_clicks*is_new_treatment>=opt.target_click).sum()/is_new_treatment.sum()
    coldstart_rate_control_group = (total_clicks*is_new_control>=opt.target_click).sum()/is_new_control.sum()
    print('Cold Start Success Rate of treat   group:', coldstart_rate_treat_group)
    print('Cold Start Success Rate of control group:', coldstart_rate_control_group)
    print('================================================')
    return (rev_new_avg_treat_group, rev_new_avg_control_group,
            rev_new_tot_treat_group, rev_new_tot_control_group,
            clicks_new_avg_treat_group, clicks_new_avg_control_group,
            clicks_new_tot_treat_group, clicks_new_tot_control_group,
            coldstart_rate_treat_group, coldstart_rate_control_group)

## 1.4 user-side exp

In [142]:
def user_exp(opt, is_new_index, target_bid_price, coefficient_ctr_model):
    print('User-side experiment')
    print('# of users in treatment condition', int(opt.total_pv*opt.pv_treatment_ratio))
    print('# of users in control condition', int(opt.total_pv*opt.pv_treatment_ratio))
    #initialization
    target_cpc = np.copy(target_bid_price)
    scaled_PID_cycle_length = int(opt.PID_cycle/opt.pv_treatment_ratio)

    total_clicks = np.zeros(opt.total_ads)#accumulated number of clicks
    total_clicks_t = np.zeros(opt.total_ads)#accumulated number of clicks in treatment UVs
    total_clicks_c = np.zeros(opt.total_ads)#accumulated number of clicks in control UVs
    total_impressions = np.zeros(opt.total_ads)#accumulated number of impressions
    total_impressions_t = np.zeros(opt.total_ads)#accumulated number of impressions in treatment UVs
    total_impressions_c = np.zeros(opt.total_ads)#accumulated number of impressions in control UVs
    total_cost = np.zeros(opt.total_ads)#accumulated cost of ads
    total_cost_t = np.zeros(opt.total_ads)
    total_cost_c = np.zeros(opt.total_ads)
    coefficient_ctr_pred_model = np.zeros([opt.total_ads, dim_context]) 

    for i in range(opt.total_ads):
        if is_new_index[i]==0:
            coefficient_ctr_pred_model[i] = coefficient_ctr_model[i]
        else:
            coefficient_ctr_pred_model[i, -1] = 0.5*coefficient_ctr_model[i, -1]+0.5*0.067
    new_ad_index = np.where(is_new_index==1)[0]

    current_cpc_control = np.zeros(opt.total_ads)
    for i in range(opt.total_ads):
        if is_new_index[i]==1:
            current_cpc_control[i] = target_cpc[i] + opt.reward/opt.target_click

    current_cpc_treat = np.zeros(opt.total_ads)
    for i in range(opt.total_ads):
        if is_new_index[i]==1:
            current_cpc_treat[i] = target_cpc[i] + (opt.reward+opt.delta)/opt.target_click


    #simulation starts
    for i in tqdm(range(opt.total_pv), desc="user side experiment"):
        ##########################generate context##########################
        contexts = context_generate()
        uv_condition = np.random.choice(['t', 'c', 'ow'], 1, 
                                        p=[opt.pv_treatment_ratio, opt.pv_treatment_ratio, 1-2*opt.pv_treatment_ratio])[0]

        ##########################calculate eCPM and Auction##########################
        #calculate pCTR and eCPM = pCTR*bid
        current_bid_np = np.zeros(opt.total_ads)
        pCTR = np.maximum(0, np.minimum(np.matmul(coefficient_ctr_pred_model, contexts), 1))
        if uv_condition == 't':
            current_bid_np = target_cpc*(1-is_new_index) + current_cpc_treat*is_new_index
        elif uv_condition == 'c':
            current_bid_np = target_cpc*(1-is_new_index) + current_cpc_control*is_new_index
        else:
            current_bid_np = target_cpc*(1-is_new_index) + current_cpc_control*is_new_index

        current_bid_np -= 10000*np.greater(total_cost, opt.budget_limit*np.ones(opt.total_ads))
        
        if uv_condition == 't':
            current_bid_np -= 10000*np.greater(total_cost_t, opt.pv_treatment_ratio*opt.budget_limit)*is_new_index
        elif uv_condition == 'c':
            current_bid_np -= 10000*np.greater(total_cost_c, opt.pv_treatment_ratio*opt.budget_limit)*is_new_index

        ecpm = pCTR * current_bid_np
        #ranking
        max_index = np.argmax(ecpm, axis=0)

        ##########################click feedback and cost reduction##########################  
        total_impressions[max_index] += 1
        if uv_condition == 't':
            total_impressions_t[max_index] += 1
        elif uv_condition == 'c':
            total_impressions_c[max_index] += 1

        #get the click feedback
        real_ctr = np.matmul(coefficient_ctr_model[max_index], contexts)
        is_clicked = np.random.binomial(1, max(0, min(real_ctr, 1)))
        if is_clicked == 1:#if clicked
            total_clicks[max_index] += 1
            total_cost[max_index] += target_cpc[max_index]
            if uv_condition == 't':
                total_clicks_t[max_index] += 1
                total_cost_t[max_index] += target_cpc[max_index]
            elif uv_condition == 'c':
                total_clicks_c[max_index] += 1
                total_cost_c[max_index] += target_cpc[max_index]


        ##########################update pCTR prediction model##########################
        if is_new_index[max_index]>0:#only update ctr model for new ads since we assume mature ad prediction is accurate
            coefficient_ctr_pred_model[max_index] += opt.l_rate*(is_clicked-pCTR[max_index])*contexts#SGD for linear models

        ##########################update lambda or PID##########################
        if i%scaled_PID_cycle_length==(scaled_PID_cycle_length-1):
            current_ecpc = total_cost/(total_clicks+0.01)#compute ecpc, +0.01 to avoid 0/0=NaN
            current_cpc_treat = PID_sys(current_cpc_treat,target_cpc,current_ecpc)
            current_cpc_treat = current_cpc_treat*is_new_index

            current_cpc_control = PID_sys(current_cpc_control,target_cpc,current_ecpc)
            current_cpc_control = current_cpc_control*is_new_index

        current_cpc_treat   = np.less(total_clicks_t, opt.pv_treatment_ratio*opt.target_click)*current_cpc_treat   + (1- np.less(total_clicks_t, opt.pv_treatment_ratio*opt.target_click))*target_cpc  
        current_cpc_control = np.less(total_clicks_c, opt.pv_treatment_ratio*opt.target_click)*current_cpc_control + (1- np.less(total_clicks_c, opt.pv_treatment_ratio*opt.target_click))*target_cpc
        
    #Simulation results
    print('================================================')
    rev_new_avg_treat_group = np.sum(total_cost_t*is_new_index)/(np.sum(is_new_index)*opt.total_pv*opt.pv_treatment_ratio)
    rev_new_avg_control_group = np.sum(total_cost_c*is_new_index)/(np.sum(is_new_index)*opt.total_pv*opt.pv_treatment_ratio)
    rev_new_tot_treat_group = np.sum(total_cost_t*is_new_index)
    rev_new_tot_control_group = np.sum(total_cost_c*is_new_index)
    print('Average Revenue in new of treat   group:', rev_new_avg_treat_group)
    print('Average Revenue in new of control group:', rev_new_avg_control_group)
    print('Total   Revenue in new of treat   group:', rev_new_tot_treat_group)
    print('Total   Revenue in new of control group:', rev_new_tot_control_group)
    
    clicks_new_avg_treat_group = np.sum(total_clicks_t*is_new_index)/(np.sum(is_new_index)*opt.total_pv*opt.pv_treatment_ratio)
    clicks_new_avg_control_group = np.sum(total_clicks_c*is_new_index)/(np.sum(is_new_index)*opt.total_pv*opt.pv_treatment_ratio)
    clicks_new_tot_treat_group = np.sum(total_clicks_t*is_new_index)
    clicks_new_tot_control_group = np.sum(total_clicks_c*is_new_index)
    print('Average clicks in new of treat   group:', clicks_new_avg_treat_group)
    print('Average clicks in new of control group:', clicks_new_avg_control_group)
    print('Total   clicks in new of treat   group:', clicks_new_tot_treat_group)
    print('Total   clicks in new of control group:', clicks_new_tot_control_group)
    
    coldstart_rate_treat_group = (total_clicks_t*is_new_index>=opt.pv_treatment_ratio*opt.target_click).sum()/is_new_index.sum()
    coldstart_rate_control_group = (total_clicks_c*is_new_index>=opt.pv_treatment_ratio*opt.target_click).sum()/is_new_index.sum()
    print('Cold Start Success Rate of treat   group:', coldstart_rate_treat_group)
    print('Cold Start Success Rate of control group:', coldstart_rate_control_group)
    print('================================================')
    return (rev_new_avg_treat_group, rev_new_avg_control_group,
            rev_new_tot_treat_group, rev_new_tot_control_group,
            clicks_new_avg_treat_group, clicks_new_avg_control_group,
            clicks_new_tot_treat_group, clicks_new_tot_control_group,
            coldstart_rate_treat_group, coldstart_rate_control_group)

## 1.5 two-side exp

In [143]:
def twoside_exp(opt, is_new_index, target_bid_price, coefficient_ctr_model):
    print('Two side randomized experiment')
    print('# of users in treatment condition', int(opt.total_pv*opt.pv_treatment_ratio))
    print('# of users in control condition', int(opt.total_pv*opt.pv_treatment_ratio))
    print('# of new ads in treatment condition', int(opt.total_ads*opt.new_ad_ratio*opt.ad_treatment_ratio))
    print('# of new ads in control condition', int(opt.total_ads*opt.new_ad_ratio*opt.ad_treatment_ratio))

    print('# of t_t pairs', int(opt.total_pv*opt.pv_treatment_ratio * opt.total_ads*opt.new_ad_ratio*opt.ad_treatment_ratio))
    print('# of t_c pairs', int(opt.total_pv*opt.pv_treatment_ratio * opt.total_ads*opt.new_ad_ratio*opt.ad_treatment_ratio))
    print('# of c_t pairs', int(opt.total_pv*opt.pv_treatment_ratio * opt.total_ads*opt.new_ad_ratio*opt.ad_treatment_ratio))
    print('# of c_c pairs', int(opt.total_pv*opt.pv_treatment_ratio * opt.total_ads*opt.new_ad_ratio*opt.ad_treatment_ratio))

    #randomly assign new ads into treatment and control groups
    is_new_treatment = np.zeros(opt.total_ads)
    is_new_control = np.zeros(opt.total_ads)
    new_ad_index = np.where(is_new_index==1)[0]
    treatment_new_ad_index = np.random.choice(new_ad_index, int(opt.total_ads*opt.new_ad_ratio*opt.ad_treatment_ratio), replace=False)

    control_new_ad_index = np.random.choice(np.setdiff1d(new_ad_index, treatment_new_ad_index), 
                                              int(opt.total_ads*opt.new_ad_ratio*opt.ad_treatment_ratio), replace=False)

    for i in range(int(opt.total_ads*opt.new_ad_ratio*opt.ad_treatment_ratio)):
        is_new_treatment[treatment_new_ad_index[i]] = 1
        is_new_control[control_new_ad_index[i]] = 1

    target_cpc = np.copy(target_bid_price)
    scaled_PID_cycle_length = int(opt.PID_cycle/opt.pv_treatment_ratio)
    mature_ad_index = np.where(is_new_index==0)[0]

    total_impressions = np.zeros(opt.total_ads)#accumulated number of impressions
    total_impressions_two_t = np.zeros(opt.total_ads)#accumulated number of impressions in treatment UVs & ads
    total_impressions_two_c = np.zeros(opt.total_ads)#accumulated number of impressions in control UVs & ads
    total_clicks = np.zeros(opt.total_ads)#accumulated number of clicks
    total_clicks_two_t = np.zeros(opt.total_ads)#accumulated number of clicks in treatment UVs & ads
    total_clicks_two_c = np.zeros(opt.total_ads)#accumulated number of clicks in control UVs & ads
    total_cost = np.zeros(opt.total_ads)#accumulated cost of ads
    total_cost_two_t = np.zeros(opt.total_ads)
    total_cost_two_c = np.zeros(opt.total_ads)

    coefficient_ctr_pred_model = np.zeros([opt.total_ads, dim_context]) 
    #assume for mature ads, we know exactly the true CTR model
    for i in range(opt.total_ads):
        if is_new_index[i]==0:
            coefficient_ctr_pred_model[i] = coefficient_ctr_model[i]
        else:
            coefficient_ctr_pred_model[i, -1] = 0.5*coefficient_ctr_model[i, -1]+0.5*0.067

    current_cpc_control = np.zeros(opt.total_ads)
    for i in range(opt.total_ads):
        if is_new_index[i]==1:
            current_cpc_control[i] = target_cpc[i] + opt.reward/opt.target_click

    current_cpc_treat = np.zeros(opt.total_ads)
    for i in range(opt.total_ads):
        if is_new_index[i]==1:
            current_cpc_treat[i] = target_cpc[i] + (opt.reward+opt.delta)/opt.target_click

    #simulation starts
    for i in tqdm(range(opt.total_pv),desc="two side experiment"):
        ##########################generate context##########################
        contexts = context_generate()
        uv_condition = np.random.choice(['t', 'c', 'ow'], 1, 
                                    p=[opt.pv_treatment_ratio, opt.pv_treatment_ratio, 1-2*opt.pv_treatment_ratio])[0]

        ##########################calculate eCPM and Auction##########################
        #calculate pCTR and eCPM = pCTR*bid
        current_bid_np = np.copy(target_cpc)
        pCTR = np.maximum(0, np.minimum(np.matmul(coefficient_ctr_pred_model, contexts), 1))
        if uv_condition == 't':
            current_bid_np = current_bid_np*(1-is_new_index) + current_cpc_treat*is_new_treatment + current_cpc_control*(is_new_index - is_new_treatment)
        elif uv_condition == 'c':
            current_bid_np = current_bid_np*(1-is_new_index) + current_cpc_control*is_new_treatment + current_cpc_control*(is_new_index - is_new_treatment)
        else:
            current_bid_np = current_bid_np*(1-is_new_index) + current_cpc_control*is_new_treatment + current_cpc_control*(is_new_index - is_new_treatment)

        current_bid_np -= 10000*np.greater(total_cost, opt.budget_limit*np.ones(opt.total_ads))
        
        if uv_condition == 't':
            current_bid_np -= 10000*np.greater(total_cost_two_t, opt.pv_treatment_ratio*opt.budget_limit)
        elif uv_condition == 'c':
            current_bid_np -= 10000*np.greater(total_cost_two_c, opt.pv_treatment_ratio*opt.budget_limit)
        
        ecpm = pCTR * current_bid_np
        max_index = 0
        max_index = np.argmax(ecpm, axis=0)

        ##########################click feedback and cost reduction##########################  
        total_impressions[max_index] += 1
        if uv_condition == 't':
            total_impressions_two_t[max_index] += 1
        elif uv_condition == 'c':
            total_impressions_two_c[max_index] += 1

        #get the feedback
        real_ctr = np.matmul(coefficient_ctr_model[max_index], contexts)
        is_clicked = np.random.binomial(1, max(0, min(real_ctr, 1)))
        if is_clicked == 1:#if clicked
            total_clicks[max_index] += 1
            total_cost[max_index] += target_cpc[max_index]
            if uv_condition == 't':
                total_clicks_two_t[max_index] += 1
                total_cost_two_t[max_index] += target_cpc[max_index]
            elif uv_condition == 'c':
                total_clicks_two_c[max_index] += 1
                total_cost_two_c[max_index] += target_cpc[max_index]


        ##########################update pCTR prediction model##########################
        if is_new_index[max_index]>0:#only update ctr model for new ads since we assume mature ad prediction is accurate
            coefficient_ctr_pred_model[max_index] += opt.l_rate*(is_clicked-pCTR[max_index])*contexts#SGD for linear models

        ##########################update lambda or PID##########################
        if i%scaled_PID_cycle_length==(scaled_PID_cycle_length-1):
            current_ecpc = total_cost/(total_clicks+0.01)#compute ecpc, +0.01 to avoid 0/0=NaN

            current_cpc_treat = PID_sys(current_cpc_treat,target_cpc,current_ecpc)
            current_cpc_treat = current_cpc_treat*is_new_index

            current_cpc_control = PID_sys(current_cpc_control,target_cpc,current_ecpc)
            current_cpc_control = current_cpc_control*is_new_index

        current_cpc_treat   = np.less(total_clicks_two_t, opt.pv_treatment_ratio*opt.target_click)*current_cpc_treat   + (1- np.less(total_clicks_two_t, opt.pv_treatment_ratio*opt.target_click))*target_cpc  
        current_cpc_control = np.less(total_clicks_two_c, opt.pv_treatment_ratio*opt.target_click)*current_cpc_control + (1- np.less(total_clicks_two_c, opt.pv_treatment_ratio*opt.target_click))*target_cpc

    #Simulation results
    print('================================================')
    rev_new_avg_t_t = np.sum(total_cost_two_t*is_new_treatment)/(np.sum(is_new_treatment)*opt.total_pv*opt.pv_treatment_ratio)
    rev_new_avg_c_c = np.sum(total_cost_two_c*is_new_control)/(np.sum(is_new_control)*opt.total_pv*opt.pv_treatment_ratio)
    rev_new_avg_t_c = np.sum(total_cost_two_t*is_new_control)/(np.sum(is_new_control)*opt.total_pv*opt.pv_treatment_ratio)
    rev_new_avg_c_t = np.sum(total_cost_two_c*is_new_treatment)/(np.sum(is_new_treatment)*opt.total_pv*opt.pv_treatment_ratio)
    
    rev_new_tot_t_t = np.sum(total_cost_two_t*is_new_treatment)
    rev_new_tot_c_c = np.sum(total_cost_two_c*is_new_control)
    rev_new_tot_t_c = np.sum(total_cost_two_t*is_new_control)
    rev_new_tot_c_t = np.sum(total_cost_two_c*is_new_treatment)
    print('Average Revenue in new of t_t group:', rev_new_avg_t_t)
    print('Average Revenue in new of c_c group:', rev_new_avg_c_c)
    print('Average Revenue in new of t_c group:', rev_new_avg_t_c)
    print('Average Revenue in new of c_t group:', rev_new_avg_c_t)
    print('Total   Revenue in new of t_t group:', rev_new_tot_t_t)
    print('Total   Revenue in new of c_c group:', rev_new_tot_c_c)
    print('Total   Revenue in new of t_c group:', rev_new_tot_t_c)
    print('Total   Revenue in new of c_t group:', rev_new_tot_c_t)
    
    clicks_new_avg_t_t = np.sum(total_clicks_two_t*is_new_treatment)/(np.sum(is_new_treatment)*opt.total_pv*opt.pv_treatment_ratio)
    clicks_new_avg_c_c = np.sum(total_clicks_two_c*is_new_control)/(np.sum(is_new_control)*opt.total_pv*opt.pv_treatment_ratio)
    clicks_new_avg_t_c = np.sum(total_clicks_two_t*is_new_control)/(np.sum(is_new_control)*opt.total_pv*opt.pv_treatment_ratio)
    clicks_new_avg_c_t = np.sum(total_clicks_two_c*is_new_treatment)/(np.sum(is_new_treatment)*opt.total_pv*opt.pv_treatment_ratio)

    clicks_new_tot_t_t = np.sum(total_clicks_two_t*is_new_treatment)
    clicks_new_tot_c_c = np.sum(total_clicks_two_c*is_new_control)
    clicks_new_tot_t_c = np.sum(total_clicks_two_t*is_new_control)
    clicks_new_tot_c_t = np.sum(total_clicks_two_c*is_new_treatment)
    print('Average clicks in new of t_t group:', clicks_new_avg_t_t)
    print('Average clicks in new of c_c group:', clicks_new_avg_c_c)
    print('Average clicks in new of t_c group:', clicks_new_avg_t_c)
    print('Average clicks in new of c_t group:', clicks_new_avg_c_t)
    print('Total   clicks in new of t_t group:', clicks_new_tot_t_t)
    print('Total   clicks in new of c_c group:', clicks_new_tot_c_c)
    print('Total   clicks in new of t_c group:', clicks_new_tot_t_c)
    print('Total   clicks in new of c_t group:', clicks_new_tot_c_t)
    
    coldstart_rate_t_t = (total_clicks_two_t*is_new_treatment>=opt.pv_treatment_ratio*opt.target_click).sum()/is_new_treatment.sum()
    coldstart_rate_c_c = (total_clicks_two_c*is_new_control>=opt.pv_treatment_ratio*opt.target_click).sum()/is_new_control.sum()
    coldstart_rate_t_c = (total_clicks_two_t*is_new_control>=opt.pv_treatment_ratio*opt.target_click).sum()/is_new_control.sum()
    coldstart_rate_c_t = (total_clicks_two_c*is_new_treatment>=opt.pv_treatment_ratio*opt.target_click).sum()/is_new_treatment.sum()
    print('Cold Start Success Rate of t_t group:', coldstart_rate_t_t)
    print('Cold Start Success Rate of c_c group:', coldstart_rate_c_c)
    print('Cold Start Success Rate of t_c group:', coldstart_rate_t_c)
    print('Cold Start Success Rate of c_t group:', coldstart_rate_c_t)
    print('================================================')
    return (rev_new_avg_t_t, rev_new_avg_c_c, rev_new_avg_t_c, rev_new_avg_c_t, 
            rev_new_tot_t_t, rev_new_tot_c_c, rev_new_tot_t_c, rev_new_tot_c_t, 
            clicks_new_avg_t_t, clicks_new_avg_c_c, clicks_new_avg_t_c, clicks_new_avg_c_t,
            clicks_new_tot_t_t, clicks_new_tot_c_c, clicks_new_tot_t_c, clicks_new_tot_c_t,
            coldstart_rate_t_t, coldstart_rate_t_t, coldstart_rate_c_c, coldstart_rate_t_c, coldstart_rate_c_t)

# 2. simulation loop

In [171]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm

parser = argparse.ArgumentParser()
#Default cold start parameters - the cold start value = reward*I(clicks>=target_click)
parser.add_argument('--total_ads', type=int, default=1000, help='total number of ads for simulation')
parser.add_argument('--new_ad_ratio', type=float, default=0.3, help='ratio of new ads')
parser.add_argument('--total_pv', type=int, default=1000000, help='loaded total page views in one cold start horizon for simulation')
parser.add_argument('--reward', type=float, default=2000000, help='reward of cold start')
parser.add_argument('--delta',  type=float, default=600000, help='delta reward of treated group')
parser.add_argument('--target_click', type=int, default=1000, help='target click')
parser.add_argument('--alpha', type=float, default=1e-3, help='target click/total_pv')
#If one wants to change total_ads, it is better to proportionally change total_pv to balance the two-side market

#Others parameters
parser.add_argument('--budget_limit', type=int, default=5e5, help='set an uniform budget limit on ads')
parser.add_argument('--PID_cycle', type=int, default=100000, help='after every # pvs, update PID')

#Parameters about experiments - sampling rate
parser.add_argument('--ad_treatment_ratio', type=float, default=0.4, help='range from 0 to 0.5')
parser.add_argument('--pv_treatment_ratio', type=float, default=0.4, help='range from 0 to 0.5')

#Parameters about model - learning rate
parser.add_argument('--l_rate', type=float, default=0.004, help='learning rage')

#Parameters about random - random seed
parser.add_argument('--seed', type=int, default = 1, help='random seed')

_StoreAction(option_strings=['--seed'], dest='seed', nargs=None, const=None, default=1, type=<class 'int'>, choices=None, required=False, help='random seed', metavar=None)

In [172]:
opt = parser.parse_args()
np.save('my_config_seed{}.npy'.format(opt.seed), opt)
print(opt.__dict__)
np.random.seed(opt.seed)
is_new_index, target_bid_price, coefficient_ctr_model = generate_ad(n = opt.total_ads, new_raito = opt.new_ad_ratio)

{'total_ads': 1000, 'new_ad_ratio': 0.3, 'total_pv': 1000000, 'reward': 2000000, 'delta': 600000, 'target_click': 1000, 'alpha': 0.001, 'budget_limit': 500000.0, 'PID_cycle': 100000, 'ad_treatment_ratio': 0.4, 'pv_treatment_ratio': 0.4, 'l_rate': 0.004, 'seed': 1}


In [157]:
# Initialize an empty DataFrame to store the results
metrecs = ["rev_new_avg","rev_new_tot","clicks_new_avg",'clicks_new_tot','coldstart_rate']
exp = ['_all_treat', '_all_control', 
       '_ad_treat', '_ad_control', 
       '_user_treat', '_user_control', 
       '_t_t', '_c_c', '_t_c', '_c_t']
columns = [i+j for i in metrecs for j in exp ]
results_df = pd.DataFrame(columns = columns)

# Run the simulation loop 10 times
for i in tqdm(range(10), desc="Simulation progress"):
    opt = parser.parse_args()
    is_new_index, target_bid_price, coefficient_ctr_model \
    = generate_ad(n = opt.total_ads, new_raito = opt.new_ad_ratio)

    rev_new_avg_all_control, rev_new_tot_all_control, \
    clicks_new_avg_all_control, clicks_new_tot_all_control, \
    coldstart_rate_all_control = all_control(opt = opt, is_new_index = is_new_index , 
                                target_bid_price = target_bid_price, 
                                coefficient_ctr_model = coefficient_ctr_model)
    
    rev_new_avg_all_treat,   rev_new_tot_all_treat,   \
    clicks_new_avg_all_treat,   clicks_new_tot_all_treat,   \
    coldstart_rate_all_treat  = all_treat(opt = opt, is_new_index = is_new_index , 
                                target_bid_price = target_bid_price, 
                                coefficient_ctr_model = coefficient_ctr_model)

    rev_new_avg_ad_treat, rev_new_avg_ad_control, \
    rev_new_tot_ad_treat, rev_new_tot_ad_control, \
    clicks_new_avg_ad_treat, clicks_new_avg_ad_control, \
    clicks_new_tot_ad_treat, clicks_new_tot_ad_control, \
    coldstart_rate_ad_treat, coldstart_rate_ad_control \
    = ad_exp(opt = opt, is_new_index = is_new_index , 
             target_bid_price = target_bid_price, 
             coefficient_ctr_model = coefficient_ctr_model) 

    rev_new_avg_user_treat, rev_new_avg_user_control, \
    rev_new_tot_user_treat, rev_new_tot_user_control, \
    clicks_new_avg_user_treat, clicks_new_avg_user_control, \
    clicks_new_tot_user_treat, clicks_new_tot_user_control, \
    coldstart_rate_user_treat, coldstart_rate_user_control \
    = user_exp(opt = opt, is_new_index = is_new_index , 
               target_bid_price = target_bid_price, 
               coefficient_ctr_model = coefficient_ctr_model) 
        
    rev_new_avg_t_t, rev_new_avg_c_c, rev_new_avg_t_c, rev_new_avg_c_t,  \
    rev_new_tot_t_t, rev_new_tot_c_c, rev_new_tot_t_c, rev_new_tot_c_t, \
    clicks_new_avg_t_t, clicks_new_avg_c_c, clicks_new_avg_t_c, clicks_new_avg_c_t, \
    clicks_new_tot_t_t, clicks_new_tot_c_c, clicks_new_tot_t_c, clicks_new_tot_c_t, \
    coldstart_rate_t_t, coldstart_rate_t_t, coldstart_rate_c_c, coldstart_rate_t_c, coldstart_rate_c_t \
    = twoside_exp(opt = opt, is_new_index = is_new_index,
                  target_bid_price = target_bid_price,
                  coefficient_ctr_model = coefficient_ctr_model) 
    # Append the results to the DataFrame
    dict_row = {}
    for j in columns:
        dict_row[j] = eval(j)
    results_df = results_df.append(dict_row, ignore_index=True)
    
    # Save the results to a CSV file after every loop
    results_df.to_csv("simulation_results_seed{}.csv".format(opt.seed), index=False)

Simulation progress:   0%|          | 0/10 [00:00<?, ?it/s]

All control experiment:   0%|          | 0/1000000 [00:00<?, ?it/s]

Average Revenue in new: 0.0196159929420868
Total   Revenue in new: 5884797.88262604
Average clicks in new: 0.00017581666666666667
Total   clicks in new: 52745.0
Cold Start Success Rate: 0.15666666666666668


All treat experiment:   0%|          | 0/1000000 [00:00<?, ?it/s]

Average Revenue in new: 0.019890403700454855
Total   Revenue in new: 5967121.110136457
Average clicks in new: 0.00019193
Total   clicks in new: 57579.0
Cold Start Success Rate: 0.17
Ad-side experiment
# of new ads in treatment condition 120
# of new ads in control condition 120


...

user side experiment:   0%|          | 0/1000000 [00:00<?, ?it/s]

Average Revenue in new of treat   group: 0.032459244853349015
Average Revenue in new of control group: 0.032193502716273234
Total   Revenue in new of treat   group: 3895109.3824018817
Total   Revenue in new of control group: 3863220.3259527883
Average clicks in new of treat   group: 0.00036599166666666667
Average clicks in new of control group: 0.00036149166666666667
Total   clicks in new of treat   group: 43919.0
Total   clicks in new of control group: 43379.0
Cold Start Success Rate of treat   group: 0.3333333333333333
Cold Start Success Rate of control group: 0.32666666666666666
Two side randomized experiment
# of users in treatment condition 400000
# of users in control condition 400000
# of new ads in treatment condition 120
# of new ads in control condition 120
# of t_t pairs 48000000
# of t_c pairs 48000000
# of c_t pairs 48000000
# of c_c pairs 48000000


two side experiment:   0%|          | 0/1000000 [00:00<?, ?it/s]

Average Revenue in new of t_t group: 0.055758200911067785
Average Revenue in new of c_c group: 0.01349859388423075
Average Revenue in new of t_c group: 0.011729835731146943
Average Revenue in new of c_t group: 0.053961179786373374
Total   Revenue in new of t_t group: 2676393.6437312537
Total   Revenue in new of c_c group: 647932.5064430761
Total   Revenue in new of t_c group: 563032.1150950532
Total   Revenue in new of c_t group: 2590136.629745922
Average clicks in new of t_t group: 0.0005026041666666667
Average clicks in new of c_c group: 0.000240625
Average clicks in new of t_c group: 0.00020045833333333333
Average clicks in new of c_t group: 0.0004824375
Total   clicks in new of t_t group: 24125.0
Total   clicks in new of c_c group: 11550.0
Total   clicks in new of t_c group: 9622.0
Total   clicks in new of c_t group: 23157.0
Cold Start Success Rate of t_t group: 0.4666666666666667
Cold Start Success Rate of c_c group: 0.225
Cold Start Success Rate of t_c group: 0.008333333333333333

C:\Users\86152\AppData\Local\Temp\ipykernel_8572\276870088.py:58: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append(dict_row, ignore_index=True)


In [173]:
parser = argparse.ArgumentParser()
#Default cold start parameters - the cold start value = reward*I(clicks>=target_click)
parser.add_argument('--total_ads', type=int, default=3000, help='total number of ads for simulation')
parser.add_argument('--new_ad_ratio', type=float, default=0.3, help='ratio of new ads')
parser.add_argument('--total_pv', type=int, default=1000000, help='loaded total page views in one cold start horizon for simulation')
parser.add_argument('--reward', type=float, default=2000000, help='reward of cold start')
parser.add_argument('--delta',  type=float, default=600000, help='delta reward of treated group')
parser.add_argument('--target_click', type=int, default=1000, help='target click')
parser.add_argument('--alpha', type=float, default=1e-3, help='target click/total_pv')
#If one wants to change total_ads, it is better to proportionally change total_pv to balance the two-side market

#Others parameters
parser.add_argument('--budget_limit', type=int, default=5e5, help='set an uniform budget limit on ads')
parser.add_argument('--PID_cycle', type=int, default=100000, help='after every # pvs, update PID')

#Parameters about experiments - sampling rate
parser.add_argument('--ad_treatment_ratio', type=float, default=0.4, help='range from 0 to 0.5')
parser.add_argument('--pv_treatment_ratio', type=float, default=0.4, help='range from 0 to 0.5')

#Parameters about model - learning rate
parser.add_argument('--l_rate', type=float, default=0.004, help='learning rage')

#Parameters about random - random seed
parser.add_argument('--seed', type=int, default = 13, help='random seed')

_StoreAction(option_strings=['--seed'], dest='seed', nargs=None, const=None, default=13, type=<class 'int'>, choices=None, required=False, help='random seed', metavar=None)

In [174]:
opt = parser.parse_args()
np.save('my_config_seed{}.npy'.format(opt.seed), opt)
print(opt.__dict__)
np.random.seed(opt.seed)
is_new_index, target_bid_price, coefficient_ctr_model = generate_ad(n = opt.total_ads, new_raito = opt.new_ad_ratio)

{'total_ads': 3000, 'new_ad_ratio': 0.3, 'total_pv': 1000000, 'reward': 2000000, 'delta': 600000, 'target_click': 1000, 'alpha': 0.001, 'budget_limit': 500000.0, 'PID_cycle': 100000, 'ad_treatment_ratio': 0.4, 'pv_treatment_ratio': 0.4, 'l_rate': 0.004, 'seed': 13}


In [175]:
# Initialize an empty DataFrame to store the results
metrecs = ["rev_new_avg","rev_new_tot","clicks_new_avg",'clicks_new_tot','coldstart_rate']
exp = ['_all_treat', '_all_control', 
       '_ad_treat', '_ad_control', 
       '_user_treat', '_user_control', 
       '_t_t', '_c_c', '_t_c', '_c_t']
columns = [i+j for i in metrecs for j in exp ]
results_df = pd.DataFrame(columns = columns)

# Run the simulation loop 10 times
for i in tqdm(range(10), desc="Simulation progress"):
    opt = parser.parse_args()
    is_new_index, target_bid_price, coefficient_ctr_model \
    = generate_ad(n = opt.total_ads, new_raito = opt.new_ad_ratio)

    rev_new_avg_all_control, rev_new_tot_all_control, \
    clicks_new_avg_all_control, clicks_new_tot_all_control, \
    coldstart_rate_all_control = all_control(opt = opt, is_new_index = is_new_index , 
                                target_bid_price = target_bid_price, 
                                coefficient_ctr_model = coefficient_ctr_model)
    
    rev_new_avg_all_treat,   rev_new_tot_all_treat,   \
    clicks_new_avg_all_treat,   clicks_new_tot_all_treat,   \
    coldstart_rate_all_treat  = all_treat(opt = opt, is_new_index = is_new_index , 
                                target_bid_price = target_bid_price, 
                                coefficient_ctr_model = coefficient_ctr_model)

    rev_new_avg_ad_treat, rev_new_avg_ad_control, \
    rev_new_tot_ad_treat, rev_new_tot_ad_control, \
    clicks_new_avg_ad_treat, clicks_new_avg_ad_control, \
    clicks_new_tot_ad_treat, clicks_new_tot_ad_control, \
    coldstart_rate_ad_treat, coldstart_rate_ad_control \
    = ad_exp(opt = opt, is_new_index = is_new_index , 
             target_bid_price = target_bid_price, 
             coefficient_ctr_model = coefficient_ctr_model) 

    rev_new_avg_user_treat, rev_new_avg_user_control, \
    rev_new_tot_user_treat, rev_new_tot_user_control, \
    clicks_new_avg_user_treat, clicks_new_avg_user_control, \
    clicks_new_tot_user_treat, clicks_new_tot_user_control, \
    coldstart_rate_user_treat, coldstart_rate_user_control \
    = user_exp(opt = opt, is_new_index = is_new_index , 
               target_bid_price = target_bid_price, 
               coefficient_ctr_model = coefficient_ctr_model) 
        
    rev_new_avg_t_t, rev_new_avg_c_c, rev_new_avg_t_c, rev_new_avg_c_t,  \
    rev_new_tot_t_t, rev_new_tot_c_c, rev_new_tot_t_c, rev_new_tot_c_t, \
    clicks_new_avg_t_t, clicks_new_avg_c_c, clicks_new_avg_t_c, clicks_new_avg_c_t, \
    clicks_new_tot_t_t, clicks_new_tot_c_c, clicks_new_tot_t_c, clicks_new_tot_c_t, \
    coldstart_rate_t_t, coldstart_rate_t_t, coldstart_rate_c_c, coldstart_rate_t_c, coldstart_rate_c_t \
    = twoside_exp(opt = opt, is_new_index = is_new_index,
                  target_bid_price = target_bid_price,
                  coefficient_ctr_model = coefficient_ctr_model) 
    # Append the results to the DataFrame
    dict_row = {}
    for j in columns:
        dict_row[j] = eval(j)
    results_df = results_df.append(dict_row, ignore_index=True)
    
    # Save the results to a CSV file after every loop
    results_df.to_csv("simulation_results_seed{}.csv".format(opt.seed), index=False)

Simulation progress:   0%|          | 0/10 [00:00<?, ?it/s]

All control experiment:   0%|          | 0/1000000 [00:00<?, ?it/s]

Average Revenue in new: 0.005100684766324836
Total   Revenue in new: 4590616.2896923525
Average clicks in new: 6.446444444444445e-05
Total   clicks in new: 58018.0
Cold Start Success Rate: 0.06111111111111111


All treat experiment:   0%|          | 0/1000000 [00:00<?, ?it/s]

Average Revenue in new: 0.00665944924593625
Total   Revenue in new: 5993504.321342625
Average clicks in new: 7.032333333333333e-05
Total   clicks in new: 63291.0
Cold Start Success Rate: 0.06222222222222222
Ad-side experiment
# of new ads in treatment condition 360
# of new ads in control condition 360


...

user side experiment:   0%|          | 0/1000000 [00:00<?, ?it/s]

Average Revenue in new of treat   group: 0.012624428813629619
Average Revenue in new of control group: 0.012530144958728702
Total   Revenue in new of treat   group: 4544794.3729066625
Total   Revenue in new of control group: 4510852.185142333
Average clicks in new of treat   group: 0.00014231944444444445
Average clicks in new of control group: 0.0001410722222222222
Total   clicks in new of treat   group: 51235.0
Total   clicks in new of control group: 50786.0
Cold Start Success Rate of treat   group: 0.1288888888888889
Cold Start Success Rate of control group: 0.12777777777777777
Two side randomized experiment
# of users in treatment condition 400000
# of users in control condition 400000
# of new ads in treatment condition 360
# of new ads in control condition 360
# of t_t pairs 144000000
# of t_c pairs 144000000
# of c_t pairs 144000000
# of c_c pairs 144000000


two side experiment:   0%|          | 0/1000000 [00:00<?, ?it/s]

Average Revenue in new of t_t group: 0.013622302499042106
Average Revenue in new of c_c group: 0.011124643602561456
Average Revenue in new of t_c group: 0.00967947709093798
Average Revenue in new of c_t group: 0.0129599953680932
Total   Revenue in new of t_t group: 1961611.5598620633
Total   Revenue in new of c_c group: 1601948.6787688497
Total   Revenue in new of t_c group: 1393844.701095069
Total   Revenue in new of c_t group: 1866239.3330054209
Average clicks in new of t_t group: 0.00019340972222222223
Average clicks in new of c_c group: 9.0125e-05
Average clicks in new of t_c group: 7.655555555555555e-05
Average clicks in new of c_t group: 0.00018197222222222223
Total   clicks in new of t_t group: 27851.0
Total   clicks in new of c_c group: 12978.0
Total   clicks in new of t_c group: 11024.0
Total   clicks in new of c_t group: 26204.0
Cold Start Success Rate of t_t group: 0.18055555555555555
Cold Start Success Rate of c_c group: 0.07777777777777778
Cold Start Success Rate of t_c gr

C:\Users\86152\AppData\Local\Temp\ipykernel_8572\27542452.py:58: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append(dict_row, ignore_index=True)
